In [4]:
from os import environ
environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.10:0.4.1 pyspark-shell' 
df = spark.read.format('xml').options(rowTag='book').load('../data/sample/book/books.xml')
df.printSchema()
df.show()

root
 |-- _id: string (nullable = true)
 |-- author: string (nullable = true)
 |-- description: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- price: double (nullable = true)
 |-- publish_date: string (nullable = true)
 |-- title: string (nullable = true)

+-----+--------------------+--------------------+---------------+-----+------------+--------------------+
|  _id|              author|         description|          genre|price|publish_date|               title|
+-----+--------------------+--------------------+---------------+-----+------------+--------------------+
|bk101|Gambardella, Matthew|An in-depth look ...|       Computer|44.95|  2000-10-01|XML Developer's G...|
|bk102|          Ralls, Kim|A former architec...|        Fantasy| 5.95|  2000-12-16|       Midnight Rain|
|bk103|         Corets, Eva|After the collaps...|        Fantasy| 5.95|  2000-11-17|     Maeve Ascendant|
|bk104|         Corets, Eva|In post-apocalyps...|        Fantasy| 5.95|  2001-03-10|   

In [5]:
df.createOrReplaceTempView("book")
query = '''
select
genre,
avg(price),
max(price),
min(price),
count(1)
from book
group by 1
order by 2 desc
'''
sqlDF = spark.sql(query)
sqlDF.show()
sqlDF.write.mode("overwrite").json("data/output/book_result.xml")

+---------------+----------+----------+----------+--------+
|          genre|avg(price)|max(price)|min(price)|count(1)|
+---------------+----------+----------+----------+--------+
|       Computer|      42.2|     49.95|     36.95|       4|
|Science Fiction|      6.95|      6.95|      6.95|       1|
|        Fantasy|      5.95|      5.95|      5.95|       4|
|        Romance|      4.95|      4.95|      4.95|       2|
|         Horror|      4.95|      4.95|      4.95|       1|
+---------------+----------+----------+----------+--------+

